In [1]:
import pandas as pd
import numpy as np
import redshift_connector

import warnings
warnings.simplefilter('ignore', pd.errors.PerformanceWarning)

from IPython.display import display, clear_output
from numba import njit

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


from ta.momentum import RSIIndicator

In [2]:
def get_unique_tokens():
    with redshift_connector.connect(
        host = 'project-poseidon.cpsnf8brapsd.us-west-2.redshift.amazonaws.com',
        database = 'token_price',
        user = 'administrator',
        password = 'Free2play2'
    ) as conn:
        with conn.cursor() as cursor:
            query = """
            WITH num_days_data AS (
                SELECT 
                    asset_id_base,
                    asset_id_quote,
                    exchange_id,
                    COUNT(*) / 24.0 AS num_days_data
                FROM token_price.coinapi.price_data_1h
                GROUP BY asset_id_base, asset_id_quote, exchange_id
            ), 
            ordered_pairs_by_data_size AS (
                SELECT
                    *,
                    ROW_NUMBER() OVER (PARTITION BY asset_id_base 
                                       ORDER BY num_days_data DESC) AS pos
                FROM num_days_data
            )

            SELECT 
                asset_id_base,
                asset_id_quote,
                exchange_id
            FROM ordered_pairs_by_data_size
            WHERE 
                pos = 1 AND
                num_days_data >= 365
            ORDER BY asset_id_base, asset_id_quote, exchange_id
            """

            # Execute query on Redshift and return result as DataFrame
            cursor.execute(query)
            df = cursor.fetch_dataframe()
            
            return df
        
def fetch_OHLCV_df_from_redshift(asset_id_base, asset_id_quote, exchange_id):
    # Connect to Redshift cluster containing price data
    with redshift_connector.connect(
        host = 'project-poseidon.cpsnf8brapsd.us-west-2.redshift.amazonaws.com',
        database = 'token_price',
        user = 'administrator',
        password = 'Free2play2'
    ) as conn:
        with conn.cursor() as cursor:
            # Query to fetch OHLCV data for a token & exchange of interest
            title = asset_id_base + '-' + asset_id_quote
            query = """
            SELECT 
                time_period_end,
                price_open,
                price_high,
                price_low,
                price_close,
                volume_traded
            FROM token_price.coinapi.price_data_1h
            WHERE 
                asset_id_base = '{}' AND
                asset_id_quote = '{}' AND
                exchange_id = '{}'
            ORDER BY time_period_start ASC
            """.format(asset_id_base, asset_id_quote, 
                        exchange_id)

            # Execute query on Redshift and return result as a DataFrame
            cursor.execute(query)

            df = cursor.fetch_dataframe().rename({'time_period_end':'Date'}, axis = 1).set_index('Date').astype(float)
            return df
        
def fetch_blockchain_metrics():
    # Connect to Redshift cluster containing price data
    with redshift_connector.connect(
        host = 'project-poseidon.cpsnf8brapsd.us-west-2.redshift.amazonaws.com',
        database = 'administrator',
        user = 'administrator',
        password = 'Free2play2'
    ) as conn:
        with conn.cursor() as cursor:
            # Query to fetch OHLCV data for a token & exchange of interest
            query = """
            WITH total_eth_sent_per_block AS (
                SELECT
                    b.block_no,
                    timestamp 'epoch' + "timestamp" * interval '1 second' AS "timestamp",
                    SUM(t.value) AS total_transaction_value
                FROM eth_data.transaction t INNER JOIN eth_data.block b
                    ON t.block_no = b.block_no
                GROUP BY
                    b.block_no,
                    b.timestamp
                ORDER BY b.block_no
            ),
            eth_usd_coinbase AS (
                SELECT 
                    time_period_start,
                    time_period_end
                FROM token_price.coinapi.price_data_1h
                WHERE
                    asset_id_base = 'ETH' AND
                    asset_id_quote = 'USD' AND
                    exchange_id = 'COINBASE'
            ),
            total_eth_sent_per_hour AS (
                SELECT
                    e.time_period_end,
                    SUM(t.total_transaction_value) AS total_eth_sent_per_hour
                FROM total_eth_sent_per_block t INNER JOIN eth_usd_coinbase e
                    ON t.timestamp >= e.time_period_start AND
                        t.timestamp < e.time_period_end 
                GROUP BY e.time_period_end
                ORDER BY e.time_period_end
            )

            SELECT *
            FROM total_eth_sent_per_hour
            """

            # Execute query on Redshift and return result
            cursor.execute(query)
            tuples = cursor.fetchall()
            
            # Return queried data as a DataFrame
            cols = ['Date', 'total_eth_send_per_hour']
            df = pd.DataFrame(tuples, columns = cols).set_index('Date')
            df = df.astype(float)

            return df

@njit
def helper(i, df, tp, sl, max_hold_time):
    if i - 1 < 0:
        return np.nan
    
    hold_countdown = max_hold_time

    for j in range(i, len(df)):
        if hold_countdown == 0:
            trade_return = (df[j] - df[i]) / df[i]
            return 1 if np.sign(trade_return) > 0 else -1
        
        price = df[j]
        price_prev = df[j - 1]
        
        if price_prev < tp and price > tp:
            return 1
        elif price_prev > sl and price < sl:
            return -1
        
        hold_countdown -= 1

    trade_return = (df[-1] - df[i]) / df[i]
    return 1 if np.sign(trade_return) > 0 else -1

@njit
def triple_barrier(df, rolling_std, max_hold_time, std_multiple_tp, std_multiple_sl):
    to_return = []

    for i in range(len(df)):
        tp = df[i] + rolling_std[i] * std_multiple_tp
        sl = df[i] - rolling_std[i] * std_multiple_sl

        if not (tp != np.nan and sl != np.nan):
            to_return.append(np.nan)
        else:
            to_return.append(helper(
                i = i, 
                df = df,
                tp = tp,
                sl = sl,
                max_hold_time = max_hold_time
            ))

    return to_return

def rolling_mean(col, window):
    return col.rolling(window).mean().rename({col.columns[0]:'rolling_mean_{}_{}'.format(col.columns[0], window)}, axis = 1)

def rolling_var(col, window):
    return col.rolling(window).var().rename({col.columns[0]:'rolling_var_{}_{}'.format(col.columns[0], window)}, axis = 1)

def rolling_skew(col, window):
    return col.rolling(window).skew().rename({col.columns[0]:'rolling_skew_{}_{}'.format(col.columns[0], window)}, axis = 1)

def rolling_kurt(col, window):
    return col.rolling(window).kurt().rename({col.columns[0]:'rolling_kurt_{}_{}'.format(col.columns[0], window)}, axis = 1)

def rrsi(close, window):
    return RSIIndicator(close = close, window = window, fillna = False).rsi().to_frame().rename({'rsi':'rsi_{}'.format(window)}, axis = 1)

def rolling_z_score(col, window):
    mean = col.rolling(window).mean().shift(1)
    std = col.rolling(window).std().shift(1)
    z = ((col - mean) / std).replace([np.inf, -np.inf], 0)

    return z.rename({col.columns[0]:'rolling_z_{}_{}'.format(col.columns[0], window)}, axis = 1)

def pca(X_train, X_test, n_components):
    ss = StandardScaler()
    pca = PCA(n_components = n_components)

    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)

    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    return X_train, X_test


In [ ]:
blockchain_metrics = fetch_blockchain_metrics()

In [4]:
data = []
triple_barrier_labels = np.array([])
returns_data = np.array([])

window = 48

for token in [{'asset_id_base': 'ETH', 'asset_id_quote': 'USD', 'exchange_id': 'COINBASE'}]:
    if token['asset_id_base'] in ['USD', 'USDT', 'USDC']:
        continue

    price_data = fetch_OHLCV_df_from_redshift(token['asset_id_base'], token['asset_id_quote'], token['exchange_id'])
    # price_data = price_data.merge(blockchain_metrics, left_index = True, right_index = True, how = 'inner')
    price_data['volume_traded'] = price_data['volume_traded'] * price_data['price_close']
    
    rolling_std = price_data.price_close.rolling(window).std().values

    triple_barrier_label = triple_barrier(
        df = price_data.price_close.values,
        rolling_std = rolling_std,
        max_hold_time = 24,
        std_multiple_tp = 2,
        std_multiple_sl = 2
    )

    returns = price_data[['price_close']].pct_change().rename({'price_close':'returns'}, axis = 1)
    volume = price_data[['volume_traded']].pct_change().rename({'volume_traded':'volume_returns'}, axis = 1)
    # eth_sent = price_data[['total_eth_send_per_hour']].pct_change().rename({'total_eth_send_per_hour':'total_eth_sent_returns'}, axis = 1)

    price_data_w_features = []

    for i in range(6, 24 + 1, 6):
        rm_price = rolling_mean(returns, i)
        rv_price = rolling_var(returns, i)
        rs_price = rolling_skew(returns, i)
        rk_price = rolling_kurt(returns, i)

        rm_volume = rolling_mean(volume, i)
        rv_volume = rolling_var(volume, i)
        rs_volume = rolling_skew(volume, i)
        rk_volume = rolling_kurt(volume, i)

        # rm_total_eth_sent = rolling_mean(eth_sent, i)
        # rv_total_eth_sent = rolling_var(eth_sent, i)
        # rs_total_eth_sent = rolling_skew(eth_sent, i)
        # rk_total_eth_sent = rolling_kurt(eth_sent, i)

        rz_price = rolling_z_score(price_data[['price_close']], i)
        rz_volume = rolling_z_score(price_data[['volume_traded']], i)
        # rz_eth_sent = rolling_z_score(price_data[['total_eth_send_per_hour']], i)
        # rz_eth_sent_returns = rolling_z_score(eth_sent, i)
        rz_returns = rolling_z_score(returns, i)
        rz_volume_returns = rolling_z_score(volume, i)

        to_comb = [
            rm_price, rv_price, rs_price, rk_price,
            rm_volume, rv_volume, rs_volume, rk_volume,
            # rm_total_eth_sent, rv_total_eth_sent, rs_total_eth_sent, rk_total_eth_sent,
            rz_price, rz_volume, rz_returns, rz_volume_returns, # rz_eth_sent, rz_eth_sent_returns,
            price_data[['price_close']], price_data[['volume_traded']], returns, volume
        ]

        comb = pd.concat(to_comb, axis = 1)
        price_data_w_features.append(comb)

    price_data_w_features = pd.concat(price_data_w_features, axis = 1)
    price_data_w_features['asset_id_base'] = token['asset_id_base']
    price_data_w_features['asset_id_quote'] = token['asset_id_quote']
    price_data_w_features['exchange_id'] = token['exchange_id']
    
    data.append(price_data_w_features)
    triple_barrier_labels = np.append(triple_barrier_labels, triple_barrier_label)
    returns_data = np.append(
        returns_data, 
        ((returns - returns.mean()) / returns.std()).shift(-1).values.tolist()
    )

data = pd.concat(data)
triple_barrier_labels = np.ravel(triple_barrier_labels)
returns = np.ravel(returns_data)

data['triple_barrier_label'] = triple_barrier_labels
data['next_period_returns'] = returns

data = data.dropna()

data.tail()

,rolling_mean_returns_6,rolling_var_returns_6,rolling_skew_returns_6,rolling_kurt_returns_6,rolling_mean_volume_returns_6,rolling_var_volume_returns_6,rolling_skew_volume_returns_6,rolling_kurt_volume_returns_6,rolling_z_price_close_6,rolling_z_volume_traded_6,...,rolling_z_volume_returns_24,price_close,volume_traded,returns,volume_returns,asset_id_base,asset_id_quote,exchange_id,triple_barrier_label,next_period_returns
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-03 10:00:00,-0.000582,0.000002,0.018924,-3.140382,0.182326,0.089533,-1.762533,3.445252,-0.410847,0.280720,...,-0.425129,1833.19,5.078954e+06,0.001021,-0.386810,ETH,USD,COINBASE,1.0,0.102083
2023-08-03 11:00:00,-0.000029,0.000002,-0.861723,-1.729962,0.054018,0.196128,-0.933031,-1.267105,0.746742,-1.479556,...,-0.636125,1835.56,1.966030e+06,0.001293,-0.612907,ETH,USD,COINBASE,1.0,-0.044103
2023-08-03 12:00:00,-0.000215,0.000002,-0.418171,-1.992191,-0.010911,0.181552,-0.429755,-1.620220,0.532232,-1.380907,...,-0.259812,1834.91,1.809348e+06,-0.000354,-0.079695,ETH,USD,COINBASE,1.0,0.507348
2023-08-03 15:00:00,0.001105,0.000007,1.439670,3.012029,0.380586,1.288564,1.744326,3.450732,6.227006,0.769473,...,1.492130,1845.66,6.400588e+06,0.005859,2.537511,ETH,USD,COINBASE,-1.0,-0.352201
2023-08-03 16:00:00,0.000754,0.000010,0.353813,2.027870,0.322926,1.306547,1.928331,4.044915,0.622522,0.713953,...,-0.230016,1838.60,6.713117e+06,-0.003825,0.048828,ETH,USD,COINBASE,1.0,0.279886


In [5]:
data

,rolling_mean_returns_6,rolling_var_returns_6,rolling_skew_returns_6,rolling_kurt_returns_6,rolling_mean_volume_returns_6,rolling_var_volume_returns_6,rolling_skew_volume_returns_6,rolling_kurt_volume_returns_6,rolling_z_price_close_6,rolling_z_volume_traded_6,...,rolling_z_volume_returns_24,price_close,volume_traded,returns,volume_returns,asset_id_base,asset_id_quote,exchange_id,triple_barrier_label,next_period_returns
Date,,,,,,,,,,,,,,,,,,,,,
2016-06-11 16:00:00,0.000370,0.000030,1.194246,1.660240,6.175443,109.913178,0.968556,-1.828696,-0.182277,-0.658244,...,-0.520602,14.01,2.543026e+03,-0.004264,-0.653405,ETH,USD,COINBASE,1.0,0.747600
2016-06-11 17:00:00,0.002632,0.000031,0.376260,-1.424781,6.219525,109.297171,0.964792,-1.829165,1.688177,-0.108947,...,-0.339406,14.13,3.832640e+03,0.008565,0.507118,ETH,USD,COINBASE,1.0,0.050147
2016-06-11 18:00:00,0.001072,0.000018,0.974791,2.165999,6.244704,108.866860,0.965707,-1.828842,1.949433,-0.923106,...,-0.539042,14.14,7.156420e+02,0.000708,-0.813277,ETH,USD,COINBASE,1.0,0.426743
2016-06-11 19:00:00,0.001660,0.000021,0.400492,-0.249068,4.877867,78.288204,1.430586,0.899403,2.678351,1.733060,...,1.158294,14.21,8.431963e+03,0.004950,10.782376,ETH,USD,COINBASE,1.0,-0.075135
2016-06-11 20:00:00,0.001780,0.000020,0.358142,0.032116,4.992294,76.767803,1.442136,0.952166,1.361458,0.707988,...,-0.493559,14.20,6.257365e+03,-0.000704,-0.257899,ETH,USD,COINBASE,1.0,-0.325212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-03 10:00:00,-0.000582,0.000002,0.018924,-3.140382,0.182326,0.089533,-1.762533,3.445252,-0.410847,0.280720,...,-0.425129,1833.19,5.078954e+06,0.001021,-0.386810,ETH,USD,COINBASE,1.0,0.102083
2023-08-03 11:00:00,-0.000029,0.000002,-0.861723,-1.729962,0.054018,0.196128,-0.933031,-1.267105,0.746742,-1.479556,...,-0.636125,1835.56,1.966030e+06,0.001293,-0.612907,ETH,USD,COINBASE,1.0,-0.044103
2023-08-03 12:00:00,-0.000215,0.000002,-0.418171,-1.992191,-0.010911,0.181552,-0.429755,-1.620220,0.532232,-1.380907,...,-0.259812,1834.91,1.809348e+06,-0.000354,-0.079695,ETH,USD,COINBASE,1.0,0.507348


In [ ]:
pct_training_data = 0.7
train_cutoff = int(len(data) * pct_training_data)

features = pd.get_dummies(data.drop(['triple_barrier_label', 'next_period_returns'], axis = 1))

X_train = features.iloc[:train_cutoff]
y_train = data['triple_barrier_label'].iloc[:train_cutoff]

X_test = features.iloc[train_cutoff:]
y_test = data['triple_barrier_label'].iloc[train_cutoff:]

X_train, X_test = pca(X_train, X_test, 0.95)

In [25]:
pct_training_data = 0.7
train_cutoff = int(len(data) * pct_training_data)

features = pd.get_dummies(data.drop(['triple_barrier_label', 'next_period_returns'], axis = 1))

X_train = features.iloc[:train_cutoff]
y_train = data['next_period_returns'].iloc[:train_cutoff]

X_test = features.iloc[train_cutoff:]
y_test = data['next_period_returns'].iloc[train_cutoff:]

In [27]:
lin = LinearRegression()
lin.fit(X_train, y_train)

y_train_pred = lin.predict(X_train)
y_test_pred = lin.predict(X_test)

print('Training Data Regression Report:')
print()
print('RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred)))
print('R2: ', r2_score(y_train, y_train_pred))
print()
print('Test Data Classification Report:')
print()
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred)))
print('R2: ', r2_score(y_test, y_test_pred))

Training Data Regression Report:

RMSE:  1.0021127971479917
R2:  -1.3207889848043664e-05

Test Data Classification Report:

RMSE:  0.9633873313272675
R2:  -1.3238382587932307e-06


In [ ]:
rf = RandomForestRegressor(n_estimators = 2, criterion = 'absolute_error')
rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print('Training Data Regression Report:')
print()
print('RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred)))
print('R2: ', r2_score(y_train, y_train_pred))
print()
print('Test Data Classification Report:')
print()
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred)))
print('R2: ', r2_score(y_test, y_test_pred))

In [ ]:
lin = GradientBoostingRegressor(n_estimators = 2, loss = 'absolute_error')
lin.fit(X_train, y_train)

y_train_pred = lin.predict(X_train)
y_test_pred = lin.predict(X_test)

print('Training Data Regression Report:')
print()
print('RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred)))
print('R2: ', r2_score(y_train, y_train_pred))
print()
print('Test Data Classification Report:')
print()
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred)))
print('R2: ', r2_score(y_test, y_test_pred))

In [32]:
clf = MLPRegressor(hidden_layer_sizes=(1000,1000,1000), max_iter = 5)
lin.fit(X_train, y_train)

y_train_pred = lin.predict(X_train)
y_test_pred = lin.predict(X_test)

print('Training Data Regression Report:')
print()
print('MSE: ', mean_squared_error(y_train, y_train_pred))
print('R2: ', r2_score(y_train, y_train_pred))
print()
print('Test Data Classification Report:')
print()
print('MSE: ', mean_squared_error(y_test, y_test_pred))
print('R2: ', r2_score(y_test, y_test_pred))

Training Data Regression Report:

MSE:  1.004230058207772
R2:  -1.3207889848043664e-05

Test Data Classification Report:

MSE:  0.9281151501618744
R2:  -1.3238382587932307e-06


In [ ]:
log = LogisticRegression(max_iter = 100, solver='saga', penalty = 'l1')
log.fit(X_train, y_train)

y_train_pred = log.predict(X_train)
y_test_pred = log.predict(X_test)

print('Training Data Classification Report:')
print()
print(classification_report(y_train, y_train_pred))
print()
print('Test Data Classification Report:')
print()
print(classification_report(y_test, y_test_pred))

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print('Training Data Classification Report:')
print()
print(classification_report(y_train, y_train_pred))
print()
print('Test Data Classification Report:')
print()
print(classification_report(y_test, y_test_pred))

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

y_train_pred = gb.predict(X_train)
y_test_pred = gb.predict(X_test)

print('Training Data Classification Report:')
print()
print(classification_report(y_train, y_train_pred))
print()
print('Test Data Classification Report:')
print()
print(classification_report(y_test, y_test_pred))

In [ ]:
clf = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(100,100), max_iter = 100)

clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

print('Training Data Classification Report:')
print()
print(classification_report(y_train, y_train_pred))
print()
print('Test Data Classification Report:')
print()
print(classification_report(y_test, y_test_pred))

In [1]:
import requests
url = 'https://rest.coinapi.io/v1/orderbooks/BITSTAMP_SPOT_BTC_USD/history?time_start=2016-01-01T00:00:00&limit=1'
headers = {'X-CoinAPI-Key' : 'E66909AC-6FEE-4E94-A06D-3E6AC7B1FF69'}
response = requests.get(url, headers=headers).json()

response


[{'symbol_id': 'BITSTAMP_SPOT_BTC_USD',
  'time_exchange': '2016-01-01T00:00:01.4430000Z',
  'time_coinapi': '2016-01-01T00:00:01.5830000Z',
  'asks': [{'price': 430.89, 'size': 0.94125434},
   {'price': 431.04, 'size': 1.50857567},
   {'price': 431.06, 'size': 0.5},
   {'price': 431.25, 'size': 1.05719},
   {'price': 431.26, 'size': 1.13740354},
   {'price': 431.41, 'size': 5.208},
   {'price': 431.7, 'size': 14.059},
   {'price': 432.11, 'size': 4.635},
   {'price': 432.36, 'size': 6.9631},
   {'price': 432.48, 'size': 13.8492},
   {'price': 432.52, 'size': 1.17},
   {'price': 432.53, 'size': 1.17742},
   {'price': 432.56, 'size': 40},
   {'price': 432.59, 'size': 17.8},
   {'price': 432.6, 'size': 17.80005534},
   {'price': 432.83, 'size': 9.2466},
   {'price': 432.84, 'size': 39.75},
   {'price': 432.88, 'size': 41.73},
   {'price': 432.9, 'size': 100},
   {'price': 433, 'size': 0.1},
   {'price': 433.17, 'size': 1.30767013},
   {'price': 433.33, 'size': 11},
   {'price': 433.5, 's